# Computing integrals in tensorflow

Tensorflow currently has no tools for numerical integration. Therefore, we will develop here some methods that will allow us to perform these calculations, using tensorflow's native functions.

In [ ]:
from math import pi
import tensorflow as tf
import matplotlib.pyplot as plt

## Riemann sums

As we did in the notebook `Computing gradients`, we will use the definition as a first approach. In this case, the definition given by Riemann.

**Definition:** A partition of the interval $(x_0, x_n)$ is a sequence of points $x_i \in (x_0, x_n)$ such that if $i < j \iff x_i < x_j$, $\forall i, j \in (0,n)$.

Then,

\begin{equation}
s(f, h) = \sum_{i=1}^n (x_{i} - x_{i-1}) f(x_i) \leq \sum_{i=0}^n (x_{i+1} - x_{i}) f(x_i) = S(f, h)
\end{equation}
where $ h $ is a partition of the interval $ (x_0, x_n) $.

In [ ]:
def left_riemann_sum(f, h):
    left_points = tf.slice(h, [0], [h.get_shape()[0] - 1])
    right_points = tf.slice(h, [1], [h.get_shape()[0] - 1])
    intervals_length = right_points - left_points
    y = f(left_points)
    return tf.reduce_sum(y * intervals_length)

In [ ]:
def right_riemann_sum(f, h):
    left_points = tf.slice(h, [0], [h.get_shape()[0] - 1])
    right_points = tf.slice(h, [1], [h.get_shape()[0] - 1])
    intervals_length = right_points - left_points
    y = f(right_points)
    return tf.reduce_sum(y * intervals_length)

## Trapezoidal rule

\begin{equation}
\int_{x_0}^{x_1} f(x) dx \approx \sum_{i=1}^n \frac{1}{2} (x_{i} - x_{i-1}) (f(x_{i}) + f(x_{i+1}))
\end{equation}

In [ ]:
def trapezoidal_rule(f, h):
    left_points = tf.slice(h, [0], [h.get_shape()[0] - 1])
    right_points = tf.slice(h, [1], [h.get_shape()[0] - 1])
    intervals_length = right_points - left_points
    left_y = f(left_points)
    right_y = f(right_points)
    interval_height = left_y + right_y
    return tf.reduce_sum(1/2 * (intervals_length) * (interval_height))

## Simpson's rule 

\begin{equation}
\int_{x_0}^{x_1} f(x) dx \approx \frac{h}{3} \big( f(x_0) + 2 \sum_{i=1}^{n - 1} f(x_{2i}) + 4 \sum_{i=1}^{n} f(x_{2i - 1}) + f(x_n) \big)
\end{equation}

In [ ]:
def simpson_rule(f, h):
    left_points = tf.slice(h, [0], [h.get_shape()[0] - 1])
    right_points = tf.slice(h, [1], [h.get_shape()[0] - 1])
    interval_length = right_points - left_points
    average_interval_lenght = tf.reduce_mean(interval_length)
    return average_interval_lenght / 3 * (
        f(h[0]) + 2 * tf.reduce_sum(f(h[1:-1:2])) + 4*tf.reduce_sum(f(h[2:-1:2])) + f(h[-1]))

In [ ]:
left_errors = []
right_errors = []
trapezoidal_errors = []
simpson_errors = []
x_0 = tf.constant(-pi/2, dtype=tf.float64)
x_1 = tf.constant(pi/2, dtype=tf.float64)
for n in range(3, 16):
    h = tf.linspace(x_0, x_1, 2**n+1)
    left_errors.append(abs(left_riemann_sum(tf.sin, h)))
    right_errors.append(abs(right_riemann_sum(tf.sin, h)))
    trapezoidal_errors.append(abs(trapezoidal_rule(tf.sin, h)))
    simpson_errors.append(abs(simpson_rule(tf.sin, h)))

In [ ]:
plt.plot(left_errors, label='left riemann sum')
plt.plot(right_errors, label='right riemann sum')
plt.plot(trapezoidal_errors, label='trapezoidal rule')
plt.plot(trapezoidal_errors, label='simpson rule')
plt.legend()